In [1]:
import nltk, re
from nltk.stem.porter import PorterStemmer
import pandas as pd



# Reference: https://ijbnpa.biomedcentral.com/articles/10.1186/s12966-017-0525-8

# Try: need to have at least 1 key verb and 1 key noun to be classified as sedentary
# instead of relying on just 1 keyword

In [2]:
# test set
test = pd.read_json('~/Documents/PhD/HBDC/FilteredTweets/sleep_tweets_2017.12.01.20.09.json', lines=True)

test

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,quoted_status_id,quoted_status_id_str,reply_count,retweet_count,retweeted,source,text,timestamp_ms,truncated,user
0,NaN,None,2017-10-29 22:43:39,"[0, 12]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,9.245877e+17,9.245877e+17,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",u inspire me https://t.co/VdECceNqpI,2017-10-29 22:43:39.375,False,"{u'follow_request_sent': None, u'profile_use_b..."
1,NaN,None,2017-10-29 22:43:39,"[0, 26]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,9.247346e+17,9.247346e+17,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",An icon of Canadian art... https://t.co/Fxk3w5...,2017-10-29 22:43:39.431,False,"{u'follow_request_sent': None, u'profile_use_b..."
2,NaN,None,2017-10-29 22:43:40,"[9, 32]","{u'user_mentions': [{u'id': 203123011, u'indic...",NaN,NaN,0,False,low,...,9.244376e+17,9.244376e+17,0,0,False,"<a href=""http://twitter.com/download/android"" ...",@CTVNews https://t.co/vcQUwe5ww6,2017-10-29 22:43:40.198,False,"{u'follow_request_sent': None, u'profile_use_b..."
3,NaN,None,2017-10-29 22:43:40,"[40, 49]","{u'user_mentions': [{u'id': 747981270, u'indic...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@rouxx21 @TalonFruitBat @Twilight_Saint &lt;33333,2017-10-29 22:43:40.311,False,"{u'follow_request_sent': None, u'profile_use_b..."
4,NaN,None,2017-10-29 22:43:39,"[0, 62]","{u'user_mentions': [{u'id': 1565023633, u'indi...",{u'media': [{u'expanded_url': u'https://twitte...,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",I get A's &amp; B's and she gets D's....nuts!!...,2017-10-29 22:43:39.434,False,"{u'follow_request_sent': None, u'profile_use_b..."
5,NaN,None,2017-10-29 22:43:40,"[27, 140]","{u'user_mentions': [{u'id': 4560764232, u'indi...",NaN,"{u'display_text_range': [27, 167], u'entities'...",0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/android"" ...",@_Nick_Nation_ @nickcarter Can believe its bee...,2017-10-29 22:43:40.770,True,"{u'follow_request_sent': None, u'profile_use_b..."
6,NaN,None,2017-10-29 22:43:40,"[0, 100]","{u'user_mentions': [{u'id': 567010052, u'indic...",{u'media': [{u'expanded_url': u'https://twitte...,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",Fav song 🔥 Honestly I’ve been playing this so...,2017-10-29 22:43:40.808,False,"{u'follow_request_sent': None, u'profile_use_b..."
7,NaN,None,2017-10-29 22:43:41,"[0, 0]","{u'user_mentions': [], u'symbols': [], u'hasht...",{u'media': [{u'expanded_url': u'https://twitte...,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/android"" ...",https://t.co/dq4Ti7e1wx,2017-10-29 22:43:41.667,False,"{u'follow_request_sent': None, u'profile_use_b..."
8,NaN,"{u'type': u'Point', u'coordinates': [-66.0196,...",2017-10-29 22:43:42,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""https://untappd.com"" rel=""nofollow"">U...",It has more sweet flavour notes than most othe...,2017-10-29 22:43:42.172,False,"{u'follow_request_sent': None, u'profile_use_b..."
9,NaN,None,2017-10-29 22:43:42,"[0, 18]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,9.247512e+17,9.247512e+17,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",Correct.That’s it. https://t.co/0jtu2oILGN,2017-10-29 22:43:42.359,False,"{u'follow_request_sent': None, u'profile_use_b..."


In [3]:
# Verbs indicate sedentary
verbs = ['watch', 'chill', 'work', 'study', 'lying', 'recline', 'sit', 'read', 'draw', 'paint'
        , 'drive']

# Nouns indicate sedentary
nouns = ['game', 'video game', 'school', 'office', 'television', 'tv', 
         'computer', 'phone', 'iphone', 'tablet', 'ipad', 'kindle', 'book', 
         'newspaper', 'homework', 'car', 'plane', 'bus', 'train', 'netflix',
         'bed', 'play', 'show'] 



# list of sport leagues
leagues = ['nba', 'worldseri', 'nfl', 'mls', 'baseball', 'mlb', 'soccer', 'mls', 'ufc', 'nhl', 'cfl']


exclusion = ['look for work', 'hire', 'job', 'career']

In [4]:
# French version
from googletrans import Translator
translator = Translator()

verbs_fr = [translator.translate(x, dest='fr').text for x in verbs]
verbs_fr

nouns_fr = [translator.translate(x, dest='fr').text for x in nouns]
nouns_fr  

[u'Jeu',
 u'jeu vid\xe9o',
 u'\xe9cole',
 u'Bureau',
 u't\xe9l\xe9vision',
 u'la t\xe9l\xe9',
 u'ordinateur',
 u't\xe9l\xe9phone',
 u'iphone',
 u'tablette',
 u'ipad',
 u'kindle',
 u'livre',
 u'journal',
 u'devoirs',
 u'car',
 u'avion',
 u'bus',
 u'train',
 u'netflix',
 u'lit',
 u'jouer',
 u'montrer']

In [5]:
# stem forms

porter = nltk.PorterStemmer()

verbs_stemmed = [porter.stem(item) for item in verbs]
#verbs_stemmed

# stemming nouns doesnt make sense
#nouns_stemmed = [porter.stem(item) for item in nouns]
#verbs_stemmed


In [6]:
# Regex pattern
verbs_regex = "\s(" + '|'.join(verbs + verbs_stemmed + verbs_fr) + ')'
verbs_regex = re.sub(' ', '\ ', verbs_regex)


nouns_regex = "\s(" + '|'.join(nouns + nouns_fr + leagues) + ')'
#nouns_regex = '\s' + '|'.join(nouns + nouns_stemmed)
#nouns_regex = re.sub(' ', '\ ', nouns_regex)


exclusion_regex = "\s(" + '|'.join(exclusion) + ')'


In [7]:
# tokenize each tweets
# look for match in keyword
# but have to include word boundary

for i, item in enumerate(test.text):
    #print item 
    lowered = item.lower()
    
    tokens = nltk.word_tokenize(lowered)

    stemmed_tokens = [porter.stem(x) for x in tokens]

    #print stemmed_tokens

    verbs_count = len(re.findall(verbs_regex, ' '.join(stemmed_tokens)))
    #print re.findall(verbs_regex, ' '.join(stemmed_tokens))
    #print verbs_count

    noun_count = len(re.findall(nouns_regex, ' '.join(stemmed_tokens)))
    #print re.findall(nouns_regex, ' '.join(stemmed_tokens))
    #print noun_count
    #print '\n'
    
    exlusion_count = len(re.findall(exclusion_regex, ' '.join(stemmed_tokens)))

    if verbs_count >=1 and noun_count >=1 and exlusion_count == 0:
    	test.at[i, 'sedentary'] = 1
    	print item
    	#print stemmed_tokens
    	print re.findall(verbs_regex, ' '.join(stemmed_tokens))
    	print re.findall(nouns_regex, ' '.join(stemmed_tokens))
    	print '\n'
    else:
    	test.at[i, 'sedentary'] = 0


Even though I slept from 8pm to 530 am and had a nap after work, I'm still exhausted and I'm going back to bed now… https://t.co/PpL1bztSny
[u'work']
[u'bed']


Okie dokie... Now that I’m just lying in bed; refusing to move because I’m warm as fuck and wrapped like a burrito.… https://t.co/I0RGGJXa7s
[u'lie']
[u'bed']


Everytime I drive my little sis somewhere she always wants to listen to Migos
[u'drive', u'lis']
[u'lit']


@Hankers @NBCNews I think Mr. McNair handled that so well.Didn't dump his player but listened to him and believed h… https://t.co/k23FkZ4nWD
[u'lis']
[u'play']


I love this Carr painting and how it shows the influence of Lawren Harris on Carr’s work and her… https://t.co/zmqsaSmXcA
[u'paint', u'work']
[u'car', u'show', u'car']


I love watching Al Jazeera Live streaming on YouTube. Always thought provoking world showcases of humanity. https://t.co/mb3lfRV1Ga
[u'watch']
[u'show']


J’suis en train de manquer sûrement le plus nice épisode de Od bali pcq je travaill

My mom hangs out in my room more than I do. She will legit lay in my bed and nap or watch Netflix
[u'watch']
[u'bed', u'netflix']


Sunday + bad weather = a good book to read @JohanCruyff 🇱🇺 #MyTurn #TotalFootball @AFCAjax @FCBarcelona https://t.co/s5sGDAdawx
[u'read']
[u'book']


THE #SNF GAME IS ON!  💯🏈

TONIGHT | #PITvsDET from the other side of the river,📍@FordField.

LISTEN ➡️ 1290 AM  #LdnOnt

#HereWeGo
#OnePride
[u'lis']
[u'game']


@Amangill89 @dkam28 @BelieveSC87 I’m good, and she’s prob working the concert or whitecaps game
[u'work']
[u'game']


@KenDryden Reading 'Game Change' in parking lots before games. At Chapter 14. U R Canada's Poet Laureate w/ @gorddownie
[u'read']
[u'game']


Too many kids to curse at Ed Hochuli the way I wanted to live, think I'll watch the #Bills game and fix that now. 
#OverOfficiousJerk
[u'watch']
[u'game']


White wine, grape Hi-Chew, peanut butter M&amp;Ms, and chocolate chip cookies. I am ready to watch Stranger Show 2 and die of fear again.
[

[u'play', u'play']


Such an unfortunate situation with Shipachyov &amp; Vegas. 
This guy can play no question. 10mns gm no PP time😟🤔
Hope he gets a chance.#Knights
[u'sit']
[u'play']


#StrangersThings2 is awesome and total #binge watched and now what do I do....#yeg #Netflix
[u'watch']
[u'netflix']


The attendance is 27,837 to watch #WhitecapsFC vs #SoundersFC in the #MLSCupPlayoffs @BCPlace. #VANvSEA
[u'watch']
[u'mls']


Gonna do a facial peel then spend the night watching King's Game👋🏾👏🏾
[u'watch']
[u'game']


U know everyone had a good night when over half the staff working walks in still lit
[u'work']
[u'lit']


#Mississauga, ON #Emploi: Personal Support Worker à VON Canada https://t.co/Xm8EjKy5Av #CommunityCare #CareGiving #Embauche
[u'work']
[u'car']


Oh no! 28.1% of my iPhone is damaged because of recent adult sites. Go to the Play Store now!

You have to admit it… https://t.co/jQzuSZdfTr
[u'sit']
[u'play']


I almost convinced myself to go to bed and not watch the rest of 

[u'train', u'bus', u'game']


I watched all of the walking dead and only missed one inning. #WorldSeries #longgame #WalkingDead #homerunderby
[u'watch']
[u'worldseri']


Fuck.
Just read it.
israelis are the most disgusting creatures on the planet. https://t.co/5YRxbPHpeM
[u'read']
[u'plane']


Brent Delgado pictured here is a pro-racist/pro-fascist that works for @MLS #MLSSoRacist #DefinitelyCrossedTheLine https://t.co/QpakM3Jfig
[u'work']
[u'mls']


@JulieSLalonde Gaslit by my whole family who is controlled by my narcissistic mother. I may have to watch this show!
[u'watch']
[u'show']


@cdnbeer I haven’t seen a TV commercial in years (refuse to watch anything live), but I can well imagine they’d be… https://t.co/HISaP5J1F6
[u'watch']
[u'tv']


@SuHolmesMcKagan Watching the show on Toronto, Ontario 🇨🇦 vidéo @JordanFPerform Thank You! @gunsnroses #GNRToronto… https://t.co/QAIoszsr8L
[u'watch']
[u'show']


I work till' 9 on Halloween meaning I can go out and get lit but I don't have a c



I promised my 4 year old son I'd pvr the rest of the game after he went to bed. Guess what we're watching till noon #WorldSeries
[u'watch']
[u'game', u'bed', u'worldseri']


is this game even real what are we even watching #WorldSeriesGame5
[u'watch']
[u'game', u'worldseri']


is this game even real?  what are we even watching?  #worldseries
[u'watch']
[u'game', u'worldseri']


I'm watching baseball. What has happened to me? #WorldSeries
[u'watch']
[u'worldseri']


Ok. Paused #StrangersThings2 watching baseball #worldseries17
[u'watch']
[u'worldseri']


This #WorldSeries is crazy &amp; I am not a baseball fan...but best vs best is fun to watch #playoffs I am gonna need bubble gum or spitz
[u'watch']
[u'worldseri', u'play']


can't remember the last time i watched anything on tv where the crowd involvement made me feel like i was there with them!! 😲⚾ #WorldSeries
[u'watch']
[u'tv', u'worldseri']


I used to play baseball and know it's boring to watch but my goodness this is damn excit

...

[u'car', u'show']


Proud to work at a company that cares so deeply for and also invests in our larger community. Team @TWG is here wit… https://t.co/fJ00lPXo4W
[u'work']
[u'car']


Loved the board @ front doors...hope to read next time I'm in the school. So much to be proud of at our little scho… https://t.co/dJFb1att15
[u'read']
[u'school', u'lit']


watch mikey go live on twitch while im watching the game🙄
[u'watch']
[u'game']


@red_pencil_man Last year I was waiting for friends to show up and there was a group of people sitting around me an… https://t.co/wZ6PFXLUp4
[u'sit']
[u'show']


Visit Knowledge Bookstore today. #reading, #books, #children, #raiseareader ,… https://t.co/cgQDp9vz2a
[u'read']
[u'book', u'book']


Figure skaters are ready for the show! https://t.co/BoOeiW6E5S
[u'read']
[u'show']


We've been working so hard😅Pls come check out the show!!! Nov 22 😁 https://t.co/zTNTDjwDUV
[u'work']
[u'show']


“Air ball! Balls back” @dallcourtney watching us play basketball
[u'wa

[u'play']


@HumboldtBroncos so nice to watch game with announcers on hockeytv. Thanks! #happyparents
[u'watch']
[u'game']


“studies have shown that addressing patients’ spiritual needs reduces stress and aids healing.”

Would love to see… https://t.co/PqXqqamfBp
[u'studi']
[u'show']


“Affordable Child Care is the number one issue facing our community.” @Kathleen_Wynne, are you going to start liste… https://t.co/w8UgHSecTu
[u'lis']
[u'car']


I need a new show to watch
[u'watch']
[u'show']


So far so good. I can watch another game! #moosevswolves #nojetsdog… https://t.co/vePPSaXCsK
[u'watch']
[u'game']


Athanasiou has been real good tonight, 1g, 1a, nice drive to the net there, showing off his speed. #LGRW
[u'drive']
[u'show']


Rams live play by play crew hard at work.  Ryerson takes an interference penalty 2 mins rams p.k. https://t.co/FLTzvUNRD2
[u'work']
[u'play', u'play']


Reminds me of my time working in India. I witnessed so much joy in the lives of those who had so little.

[u'watch']
[u'play']


I came to see my dad before i go home to get some sleep or probably laze  around and watch the touken ranbu stage play
[u'watch']
[u'play']


me playing 2k while my girlfriend is watching make up tutorials on youtube https://t.co/1ga1M4fUh4
[u'watch']
[u'play']


Getting funky and soul with @bustyandthebass at @BabylonOttawa @spectrasonic @listenharderpr… https://t.co/1GEtroZkh5
[u'lis']
[u'bus']


Ben Simmons has the potential to be a great NBA player. Watching him, Embiid, and Covington come together is already a ton of fun.
[u'watch']
[u'nba', u'play']


@ImranKhanPTI- So sad to see everyone in world working on their economy infrastructure business textiles exports im… https://t.co/17Cg2rknY4
[u'work']
[u'bus']


I just finished reading “The Boy Who Harnessed the Wind” by William K and Bryan M. Very inspiring stuff. Loved it!! #bookclubreads
[u'read']
[u'book']


@JakeHubley You are bragging about lying about being a reporter, a cowardly move that threatens re

@stephenhatesyou Guess you did not read my reply M8.▶️ Not talking about a player. The Quote is talking about 4 clu… https://t.co/CKRFT9Gn1x
[u'read']
[u'play']


The work of generalist physicians in acute &amp; community care supported by @JAMANetwork study. A model for saving Can… https://t.co/Bzd1rchYvA
[u'work', u'studi']
[u'car']


Lifting, eating well, reading books, great conversations with friends. It's almost winter, get cozy. https://t.co/8Bw0MvOLXx
[u'read']
[u'book']


We're busy putting together the #newsletter today! You'll be receiving it in the #post soon! For now, you can read… https://t.co/JhZMPB5RHS
[u'read']
[u'bus']


@Kage_99 @rickmercer Awesome piece. Been watching this show for years and  I had no idea that Rick was gay!
[u'watch']
[u'show']


@NBCBlacklist I watched the portion showing Tom’s  body being slid into the morgue’s refrigerated locker three time… https://t.co/YruGM77zpU
[u'watch']
[u'show']


This book is a crown jewels of the series, the culmination


RED HOT LISTING
216 Dufferin Street, #Lunenburg | MLS® #201727869 | $585,000
So much space for so much living!
Cont… https://t.co/77XdOdcJXE
[u'lis']
[u'mls']


@LisetteBrodey Athlete in Training. Full steam a head ... Chug a choo choo 🍻🍻😉😇 https://t.co/ZIYzjFGTbx
[u'lis']
[u'train']


That's love: @selenagomez and her dog watching @justinbieber play a little pick-up hockey last night in L.A. https://t.co/fh5RaTEtc5
[u'watch']
[u'play', u'lit']


Changing it up a little.

#pullups #exercise #workouts #workout #fitness #fitnessjunkie… https://t.co/r2USAR4xSf
[u'work', u'work']
[u'lit']


@DSIG_Samantha Morning Happy Thurs their last game on road then they are home on the weekend very exciting u watchi… https://t.co/8nwhkGEHvA
[u'watch']
[u'game']


Yay! Can't wait to watch this event at the pyeongchang winter games #HK95240 https://t.co/7YpFqUloA1
[u'watch']
[u'game']


@DSIG_Katie Morning Happy Thurs u watching the game tonight the last away one and their back on the weekend excitin… 

In [8]:
test[test.sedentary == 1]

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,quoted_status_id_str,reply_count,retweet_count,retweeted,source,text,timestamp_ms,truncated,user,sedentary
133,NaN,None,2017-10-29 22:45:35,"[0, 140]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,"{u'display_text_range': [0, 123], u'entities':...",0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/android"" ...",Even though I slept from 8pm to 530 am and had...,2017-10-29 22:45:35.606,True,"{u'follow_request_sent': None, u'profile_use_b...",1.0
156,NaN,None,2017-10-29 22:46:01,"[0, 140]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,"{u'display_text_range': [0, 138], u'entities':...",0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",Okie dokie... Now that I’m just lying in bed; ...,2017-10-29 22:46:01.853,True,"{u'follow_request_sent': None, u'profile_use_b...",1.0
270,NaN,None,2017-10-29 22:47:52,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",Everytime I drive my little sis somewhere she ...,2017-10-29 22:47:52.519,False,"{u'follow_request_sent': None, u'profile_use_b...",1.0
318,NaN,None,2017-10-29 22:49:15,"[18, 140]","{u'user_mentions': [{u'id': 17735910, u'indice...",NaN,"{u'display_text_range': [18, 158], u'entities'...",0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",@Hankers @NBCNews I think Mr. McNair handled t...,2017-10-29 22:49:15.317,True,"{u'follow_request_sent': None, u'profile_use_b...",1.0
400,NaN,"{u'type': u'Point', u'coordinates': [-123.1208...",2017-10-29 22:50:28,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://instagram.com"" rel=""nofollow"">...",I love this Carr painting and how it shows the...,2017-10-29 22:50:28.665,False,"{u'follow_request_sent': None, u'profile_use_b...",1.0
430,NaN,None,2017-10-29 22:50:56,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",I love watching Al Jazeera Live streaming on Y...,2017-10-29 22:50:56.853,False,"{u'follow_request_sent': None, u'profile_use_b...",1.0
457,NaN,None,2017-10-29 22:51:20,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",J’suis en train de manquer sûrement le plus ni...,2017-10-29 22:51:20.998,False,"{u'follow_request_sent': None, u'profile_use_b...",1.0
535,NaN,None,2017-10-29 22:52:20,NaN,"{u'user_mentions': [{u'id': 21672310, u'indice...",NaN,NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...","Yes., these are professional race car drivers ...",2017-10-29 22:52:20.093,False,"{u'follow_request_sent': None, u'profile_use_b...",1.0
647,NaN,None,2017-10-29 22:54:18,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",I hate it when am watching a trailer on TV 📺 ...,2017-10-29 22:54:18.797,False,"{u'follow_request_sent': None, u'profile_use_b...",1.0
745,NaN,None,2017-10-29 22:55:39,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...","Nice drive, great play calling, much needed. #...",2017-10-29 22:55:39.754,False,"{u'follow_request_sent': None, u'profile_use_b...",1.0
